In [16]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [17]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")
# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,afaahiti,-17.7500,-149.2833,298.08,85,100,4.71,PF,1712586949
1,1,port-aux-francais,-49.3500,70.2167,279.96,86,41,12.63,TF,1712586903
2,2,holualoa,19.6228,-155.9522,294.06,83,100,3.60,US,1712586771
3,3,albany,42.6001,-73.9662,285.52,52,79,0.89,US,1712586385
4,4,iqaluit,63.7506,-68.5145,257.00,65,20,9.77,CA,1712586612


In [18]:

map_plot = city_data_df.hvplot.points('Lng', 'Lat', geo=True, tiles='EsriImagery', size="Humidity", color='City',hover_cols=['City'])
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

In [19]:
filtered_df = city_data_df[(city_data_df['Max Temp'] > 277) & 
                           (city_data_df['Max Temp'] < 290) & 
                           (city_data_df['Wind Speed'] < 3.5) & 
                           (city_data_df['Cloudiness'] < 5)]

# Display the filtered DataFrame
filtered_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
23,23,bethel,41.3712,-73.4140,288.45,41,0,2.57,US,1712586907
96,96,newton,42.3370,-71.2092,289.25,34,0,2.57,US,1712586921
112,112,massena,44.9281,-74.8919,285.43,50,0,1.54,US,1712586785
156,156,cochrane,51.1834,-114.4687,278.63,48,0,1.54,CA,1712586907
253,253,lewistown,40.5992,-77.5714,284.35,73,0,0.00,US,1712586858
292,292,broome,42.2506,-75.8330,286.04,53,0,2.57,US,1712587075
322,322,middlebury (village),44.0153,-73.1673,286.61,54,0,1.54,US,1712587087
325,325,liminka,64.8099,25.4155,278.19,81,0,3.09,FI,1712587089
338,338,forster,-32.1667,152.5167,288.87,100,0,2.73,AU,1712587095


In [20]:
hotel_df = filtered_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']]
hotel_df['Hotel Name'] = ''
hotel_df

C:\Users\Sezy\AppData\Local\Temp\ipykernel_26704\1284306913.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Name'] = ''


,City,Country,Lat,Lng,Humidity,Hotel Name
23,bethel,US,41.3712,-73.4140,41,
96,newton,US,42.3370,-71.2092,34,
112,massena,US,44.9281,-74.8919,50,
156,cochrane,CA,51.1834,-114.4687,48,
253,lewistown,US,40.5992,-77.5714,73,
292,broome,US,42.2506,-75.8330,53,
322,middlebury (village),US,44.0153,-73.1673,54,
325,liminka,FI,64.8099,25.4155,81,
338,forster,AU,-32.1667,152.5167,100,


In [21]:
# params= {
#     "type": "accommodation.hotel",
#     "filter": f"circle:{lon},{lat},{radius}",
#     "apiKey": geoapify_key



In [22]:
print(geoapify_key)

cf01186e194644b281f9c5e22655bfcd


In [23]:
# base_url = f"https://api.geoapify.com/v2/places?"
# response = requests.get(base_url,params)
# print(response.content)

In [15]:
# Set parameters to search for a hotel
radius = 10000  # Define the radius for the search circle (in meters)
params = {
    "type": "accommodation.hotel",  # Specify the type of place to search for
    "apiKey": geoapify_key  # Replace "YOUR_API_KEY" with your actual Geoapify API key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude and longitude from the DataFrame
    lat = row["Lat"]
    lon = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)
    name_address = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
bethel - nearest hotel: Hampton Inn Danbury
newton - nearest hotel: Four Points by Sheraton
massena - nearest hotel: Quality Inn
cochrane - nearest hotel: Rockyview Hotel
lewistown - nearest hotel: No hotel found
broome - nearest hotel: No hotel found
middlebury (village) - nearest hotel: No hotel found
liminka - nearest hotel: Hotelli-Ravintola Liminganlahti
forster - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
23,bethel,US,41.3712,-73.4140,41,Hampton Inn Danbury
96,newton,US,42.3370,-71.2092,34,Four Points by Sheraton
112,massena,US,44.9281,-74.8919,50,Quality Inn
156,cochrane,CA,51.1834,-114.4687,48,Rockyview Hotel
253,lewistown,US,40.5992,-77.5714,73,No hotel found
292,broome,US,42.2506,-75.8330,53,No hotel found
322,middlebury (village),US,44.0153,-73.1673,54,No hotel found
325,liminka,FI,64.8099,25.4155,81,Hotelli-Ravintola Liminganlahti
338,forster,AU,-32.1667,152.5167,100,No hotel found


In [1]:
# the correlation with these hotels is funny
map_plot = hotel_df.hvplot.points('Lng', 'Lat', geo=True, tiles='CartoLight', size=160, color='City',
                                   hover_cols=['City', 'Hotel Name', 'Country'], frame_width=700, frame_height=500)

# Display the plot
map_plot

NameError: name 'hotel_df' is not defined